In [ ]:
from source_files import *

import math
import numpy as np
import pandas as pd
import numba

In [ ]:
sd_difference_values = sfm_snow_depth_values - aso_snow_depth_values

@numba.vectorize
def snow_volume(snow_depth, x_resolution, y_resolution):
    if snow_depth < 0:
        return 0
    else:
        return math.fabs(x_resolution) * math.fabs(y_resolution) * snow_depth

In [ ]:
df = pd.DataFrame({
        'aso_snow_depth': aso_snow_depth_values.ravel(),
        'sfm_snow_depth': sfm_snow_depth_values.ravel(),
        'sd_difference': sd_difference_values.ravel(),
        'aso_snow_volume': snow_volume(
            aso_snow_depth_values.ravel(), 
            aso_snow_depth.x_resolution, 
            aso_snow_depth.y_resolution
        ),
        'sfm_snow_volume': snow_volume(
            sfm_snow_depth_values.ravel(), 
            sfm_snow_depth.x_resolution, 
            sfm_snow_depth.y_resolution
        ),
        'casi_class': casi_classification.ravel(),
    },
)
df.dropna(how='all', inplace=True, subset=['aso_snow_depth', 'sfm_snow_depth'])
df['casi_class'] = pd.cut(df['casi_class'], CASI_MAPPING, labels=CASI_CLASSES)

In [ ]:
positive_sfm = df.query('sfm_snow_depth >= 0')
negative_sfm = df.query('sfm_snow_depth < 0')

columns = ['casi_class', 'aso_snow_depth', 'sfm_snow_depth', 'sd_difference']

pd.set_option('display.float_format', lambda x: '%.2f m' % x)

# Snow Depth

In [ ]:
df[columns].agg([np.mean, np.median, np.std])

In [ ]:
df[columns].groupby('casi_class').agg([np.mean, np.median, np.std])

### Positive SfM values

In [ ]:
positive_sfm[columns].agg([np.mean, np.median, np.std])

In [ ]:
positive_sfm[columns].groupby('casi_class').agg([np.mean, np.median, np.std])

### Negative SfM values

In [ ]:
negative_sfm[columns].groupby('casi_class').agg([np.mean, np.median, np.std])

# Snow Volume

In [ ]:
m2_style = "{:,.0f} m<sup>3</sup>"
table_style = {
    'sfm_snow_volume': m2_style,
    'aso_snow_volume': m2_style,
    'difference': m2_style,
    'sfm % to aso': "{:.2%}",
    'percent_sfm_scene': "{:.2%}",
}
columns = ['casi_class', 'aso_snow_volume', 'sfm_snow_volume']

In [ ]:
total_volume = pd.DataFrame({
    'sfm_snow_volume': df.sfm_snow_volume.sum(),
    'aso_snow_volume': df.aso_snow_volume.sum(),
    },
    index=[0]
)
total_volume['difference'] = total_volume.aso_snow_volume - total_volume.sfm_snow_volume
total_volume['sfm % to aso'] = total_volume.sfm_snow_volume / total_volume.aso_snow_volume
total_volume.style.format(table_style)

In [ ]:
num_pixels_aso = df.aso_snow_depth.count()
num_pixels_sfm = positive_sfm.aso_snow_depth.count()

In [ ]:
print(f"Percent ASO pixels with values in SfM: {num_pixels_sfm/num_pixels_aso:.2%}")

In [ ]:
pixel_stats = positive_sfm[columns].groupby('casi_class').count()

total_volume = df[columns].groupby('casi_class').sum()
total_volume['difference'] = total_volume.aso_snow_volume - total_volume.sfm_snow_volume
total_volume['sfm % to aso'] = total_volume.sfm_snow_volume / total_volume.aso_snow_volume
total_volume['percent_sfm_scene'] = (pixel_stats.sfm_snow_volume / num_pixels_sfm)
total_volume.style.format(table_style)

In [ ]:
total_volume = pd.DataFrame({
    'sfm_snow_volume': positive_sfm.sfm_snow_volume.sum(),
    'aso_snow_volume': positive_sfm.aso_snow_volume.sum(),
    },
    index=[0]
)
total_volume['difference'] = total_volume.aso_snow_volume - total_volume.sfm_snow_volume
total_volume['sfm % to aso'] = total_volume.sfm_snow_volume / total_volume.aso_snow_volume
total_volume.style.format(table_style)